
**Every single word that is input to a transformer is going to have 3 vectors.** 

Query: What I'm Looking for?[ sequence length * dk]

Key: What I can offer? [sequence length * dk]

Value: What I actually offer? [sequence length * dv]


In [2]:
import numpy as np
import math

# Here input sentence is : "My name is Yukesh"
# In the original transformer paper the original vecor embedding is 512 , but here we take the size of 8.
L, d_k, d_v= 4, 8, 8

q= np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v= np.random.randn(L, d_v)

In [4]:
print("Query: ", q)
print("Key: ", k)
print("Value: ", v)

Query:  [[ 0.8923684  -0.56333787  0.68489984  1.39229119 -0.00955662  0.15714376
  -0.63275895 -0.81128324]
 [-1.12822895  0.25969361  0.36959156 -0.0172476  -0.31501173  0.2019182
  -1.00842141 -0.22172332]
 [ 1.85105337 -0.75598807  1.18565333  2.54005835  2.50852619  1.00212263
  -2.03953596  1.6728376 ]
 [-0.8214566  -1.31764276 -0.39958134  0.61348749  1.46312925  1.54646077
  -0.40640202  0.12480838]]
Key:  [[ 1.12851897 -0.7861496   0.10245969  1.75488406  2.62988238  0.27516328
  -0.1657441   0.71910634]
 [-1.56512876  0.3563529   1.84003039  0.81677392 -1.26526623 -0.34439344
  -0.72222409  0.52305013]
 [ 0.51190939  2.61045731  0.7003261  -0.25085084 -0.75316026 -0.26655768
  -0.30322733 -1.55906172]
 [ 1.94562659  0.16961366  1.10900622 -0.17459686  0.58646725 -2.39015032
   0.26992536 -1.2967811 ]]
Value:  [[ 0.10943374  1.7869226  -1.87247558  1.66625975 -0.20259648  0.20012934
  -0.48231254  0.34872797]
 [-0.01871472  1.21117611  0.94117532 -1.45932908  0.20307052  0.705

In [5]:
np.matmul(q, k.T)

array([[ 3.50299131,  0.79062948,  0.53865837,  2.65719163],
       [-2.23497117,  3.46570669,  1.19842204, -2.39020474],
       [15.67612349, -0.08134187, -4.97879453,  0.70076153],
       [ 5.57498968, -1.44306631, -5.87943616, -5.48173286]])

Self Attention= softmax(Q. K_t/ sqrt(d_k)+ M)* V

In [7]:
# why we need sqrt(d_k) in denominator?
# ---> To reduce Variance

In [17]:
# Before dividing
print(f"The varience before dividing of \n query is {q.var()}, \n key is {k.var()} \n value is {np.matmul(q, k.T).var()} ")

#After dividing
scaled= np.matmul(q, k.T)/ math.sqrt(d_k)
print(f"\n\nThe varience after dividing of \n query is {q.var()}, \n key is {k.var()} \n value is {scaled.var()} ")

The varience before dividing of 
 query is 1.2203006500310023, 
 key is 1.3792333167950637 
 value is 25.2985192375832 


The varience after dividing of 
 query is 1.2203006500310023, 
 key is 1.3792333167950637 
 value is 3.162314904697899 


In [21]:
mask= np.tril(np.ones((L, L)))  # creating a lower triangular matrix for masking purpose
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [22]:
mask[mask==0]= -np.infty
mask[mask==1]= 0
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [23]:
scaled+ mask

array([[ 1.23849445,        -inf,        -inf,        -inf],
       [-0.79018164,  1.22531235,        -inf,        -inf],
       [ 5.54234661, -0.02875869, -1.76026969,        -inf],
       [ 1.97105651, -0.51020099, -2.07869459, -1.93808524]])

In [24]:
# applying softmax

def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

attention= softmax(scaled + mask)
attention

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.17585727e-01, 8.82414273e-01, 0.00000000e+00, 0.00000000e+00],
       [9.95539937e-01, 3.78929482e-03, 6.70768601e-04, 0.00000000e+00],
       [8.91963264e-01, 7.46020103e-02, 1.55439867e-02, 1.78907390e-02]])

In [25]:
new_v= np.matmul(attention, v)
new_v

array([[ 0.10943374,  1.7869226 , -1.87247558,  1.66625975, -0.20259648,
         0.20012934, -0.48231254,  0.34872797],
       [-0.00364629,  1.27887568,  0.61033013, -1.09180444,  0.15536987,
         0.64575934, -0.43729112,  0.58834984],
       [ 0.10803579,  1.78375588, -1.86091638,  1.65440239, -0.20028551,
         0.20175653, -0.4818298 ,  0.35065549],
       [ 0.08201201,  1.68839163, -1.61949562,  1.42313651, -0.1354339 ,
         0.2450028 , -0.43753791,  0.38119362]])

In [26]:
v

array([[ 0.10943374,  1.7869226 , -1.87247558,  1.66625975, -0.20259648,
         0.20012934, -0.48231254,  0.34872797],
       [-0.01871472,  1.21117611,  0.94117532, -1.45932908,  0.20307052,
         0.70514157, -0.43129181,  0.62028055],
       [-1.25072524,  0.31838991, -0.53452599,  1.64601169,  0.95096831,
        -0.22691941, -0.05086522,  1.68827473],
       [ 0.29280784, -0.04376056, -0.62720102,  1.12780828,  0.85761739,
         0.97352223,  1.43275447, -0.13278464]])